In [1]:
import os
import glob
import random
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from skimage.morphology import remove_small_objects, binary_opening, disk
from skimage.measure import label, regionprops
from scipy.ndimage import binary_fill_holes, distance_transform_edt
from skimage.filters import threshold_otsu, gaussian
from skimage.exposure import rescale_intensity, equalize_adapthist

# --- Variáveis Globais (do segmentation.ipynb) ---

# Caminho para o diretório de dados
DATA_DIR = '../database/axl/'
CSV_PATH = '../database/oasis_longitudinal_demographic.csv'

# Parâmetros de segmentação
N_CLUSTERS = 4
MIN_AREA_VENTRICLE = 100
MAX_AREA_VENTRICLE = 15000
MIN_AREA_BRAIN = 5000

# Seed para reprodutibilidade
random.seed(64)

print("Célula 1 executada: Bibliotecas importadas e CAMINHOS CORRIGIDOS.")

Célula 1 executada: Bibliotecas importadas e CAMINHOS CORRIGIDOS.


In [2]:
def segmentar_ventriculos(image_slice, n_clusters, min_area_brain, min_area_ventricle, max_area_ventricle):
    """
    Segmenta os ventrículos cerebrais usando a lógica robusta 
    (CLAHE, Skull Stripping, K-Means + Filtro de Área).
    """
    shape = image_slice.shape
    
    # 1. Pré-processamento
    img_norm = rescale_intensity(image_slice, out_range=(0, 1))
    img_clahe = equalize_adapthist(img_norm)
    img_smooth = gaussian(img_clahe, sigma=1)

    # 2. Remoção do Crânio
    t = threshold_otsu(img_smooth)
    mask_cerebro = img_smooth > t
    mask_cerebro = binary_opening(mask_cerebro, disk(3))
    mask_cerebro = remove_small_objects(mask_cerebro, min_size=min_area_brain) 
    mask_cerebro = binary_fill_holes(mask_cerebro)
    
    labels_cerebro = label(mask_cerebro)
    if labels_cerebro.max() == 0:
        return np.zeros(shape, dtype=bool) # Retorna máscara vazia
        
    maior_comp_label = np.argmax([region.area for region in regionprops(labels_cerebro)]) + 1
    mask_cerebro = (labels_cerebro == maior_comp_label)
    img_sem_cranio = img_smooth * mask_cerebro

    # 3. K-Means
    pixels_cerebro = img_sem_cranio[mask_cerebro].reshape(-1, 1)
    if pixels_cerebro.shape[0] < n_clusters:
        return np.zeros(shape, dtype=bool)

    kmeans = KMeans(n_clusters=n_clusters, random_state=64, n_init=10).fit(pixels_cerebro)
    centers = kmeans.cluster_centers_.flatten()
    labels_flat = kmeans.labels_

    # 4. Identificação do LCR
    sorted_indices = np.argsort(centers)
    indice_lcr = sorted_indices[0] 
    labels_kmeans = np.zeros(shape, dtype=int)
    labels_kmeans[mask_cerebro] = labels_flat + 1
    mask_lcr_total = (labels_kmeans == (indice_lcr + 1))

    # 5. Isolamento dos Ventrículos (Filtro de Área)
    labels_lcr = label(mask_lcr_total)
    regioes_lcr = regionprops(labels_lcr)
    mask_ventriculos_final = np.zeros(shape, dtype=bool)
    
    if not regioes_lcr:
        return np.zeros(shape, dtype=bool)

    for r in regioes_lcr:
        if r.area > min_area_ventricle and r.area < max_area_ventricle:
            mask_ventriculos_final[labels_lcr == r.label] = True

    return binary_fill_holes(mask_ventriculos_final)

print("Célula 2 executada: Função 'segmentar_ventriculos' definida.")

Célula 2 executada: Função 'segmentar_ventriculos' definida.


In [3]:
def extrair_features(ventricle_mask):
    """
    Calcula os descritores de forma para a máscara dos ventrículos.
    Lida com múltiplos componentes (ex: ventrículos laterais) somando
    suas áreas e perímetros, e tirando médias ponderadas para 
    descritores de forma.
    """
    
    # Se a máscara estiver vazia, retorna zero para todas as features
    if np.sum(ventricle_mask) == 0:
        return {
            'Ventricle_Area': 0,
            'Ventricle_Perimeter': 0,
            'Ventricle_Circularity': 0,
            'Ventricle_Eccentricity': 0,
            'Ventricle_Solidity': 0,
            'Ventricle_MajorAxisLength': 0
        }

    # Rotula os componentes (ex: ventrículo esquerdo e direito)
    labels = label(ventricle_mask)
    props = regionprops(labels)

    # Inicializa variáveis
    total_area = 0
    total_perimeter = 0
    
    # Para médias ponderadas
    weighted_ecc = 0
    weighted_solidity = 0
    weighted_major_axis = 0

    for prop in props:
        total_area += prop.area
        total_perimeter += prop.perimeter
        
        # Pondera a feature pela área do componente
        weighted_ecc += prop.eccentricity * prop.area
        weighted_solidity += prop.solidity * prop.area
        weighted_major_axis += prop.major_axis_length * prop.area

    # Evita divisão por zero
    if total_area == 0:
        return { 'Ventricle_Area': 0, 'Ventricle_Perimeter': 0, 'Ventricle_Circularity': 0,
                 'Ventricle_Eccentricity': 0, 'Ventricle_Solidity': 0, 'Ventricle_MajorAxisLength': 0 }

    # Calcula a Circularidade (Form Factor): 4*pi*A / P^2
    # (Evita divisão por zero se o perímetro for 0)
    if total_perimeter == 0:
        circularity = 0
    else:
        circularity = (4 * np.pi * total_area) / (total_perimeter ** 2)
        
    # Calcula as médias ponderadas
    avg_ecc = weighted_ecc / total_area
    avg_solidity = weighted_solidity / total_area
    avg_major_axis = weighted_major_axis / total_area

    return {
        'Ventricle_Area': total_area,
        'Ventricle_Perimeter': total_perimeter,
        'Ventricle_Circularity': circularity,
        'Ventricle_Eccentricity': avg_ecc,
        'Ventricle_Solidity': avg_solidity,
        'Ventricle_MajorAxisLength': avg_major_axis
    }

print("Célula 3 executada: Função 'extrair_features' definida.")

Célula 3 executada: Função 'extrair_features' definida.


In [4]:
print(f"Iniciando processamento em lote de todos os arquivos em {DATA_DIR}...")

all_files = glob.glob(os.path.join(DATA_DIR, "*.nii.gz"))
total_files = len(all_files)
results_list = []

for i, file_path in enumerate(all_files):
    
    # --- 1. Carregar Imagem ---
    try:
        # Extrai o nome base do arquivo, ex: "OAS2_0090_MR3_axl"
        base_name = os.path.basename(file_path).split('.')[0]

        # Remove o sufixo '_axl' E remove quaisquer espaços em branco
        mr_id = base_name.replace('_axl', '').strip()
        
        nii_img = nib.load(file_path)
        data = nii_img.get_fdata()
        
        if data.ndim == 3:
            slice_z = data.shape[2] // 2
            image_slice = data[:, :, slice_z]
        elif data.ndim == 2:
            image_slice = data
        else:
            print(f"AVISO: Ignorando {mr_id} (dimensão {data.ndim}D).")
            continue
            
        # --- 2. Segmentar Ventrículos ---
        ventricle_mask = segmentar_ventriculos(image_slice, 
                                               N_CLUSTERS, 
                                               MIN_AREA_BRAIN, 
                                               MIN_AREA_VENTRICLE,
                                               MAX_AREA_VENTRICLE)
        
        # --- 3. Extrair Features ---
        features = extrair_features(ventricle_mask)
        
        # --- 4. Salvar Resultado ---
        features['MRI ID'] = mr_id # Agora 'mr_id' está limpo
        results_list.append(features)
        
        if (i + 1) % 50 == 0 or (i + 1) == total_files:
             print(f"Processado {i + 1}/{total_files}: {mr_id}")

    except Exception as e:
        print(f"ERRO ao processar {base_name}: {e}")

print(f"\nProcessamento concluído. {len(results_list)} imagens analisadas.")

# Converter a lista de resultados em um DataFrame
df_features = pd.DataFrame(results_list)

# (Opcional, mas recomendado) Limpa a coluna de chave no df_features também
if not df_features.empty:
    df_features['MRI ID'] = df_features['MRI ID'].str.strip()

print("\nPré-visualização dos dados de features extraídos:")
print(df_features.head())

Iniciando processamento em lote de todos os arquivos em ../database/axl/...
Processado 50/373: OAS2_0070_MR3
Processado 50/373: OAS2_0070_MR3
Processado 100/373: OAS2_0108_MR2
Processado 100/373: OAS2_0108_MR2
Processado 150/373: OAS2_0140_MR2
Processado 150/373: OAS2_0140_MR2
Processado 200/373: OAS2_0068_MR2
Processado 200/373: OAS2_0068_MR2
Processado 250/373: OAS2_0080_MR3
Processado 250/373: OAS2_0080_MR3
Processado 300/373: OAS2_0020_MR3
Processado 300/373: OAS2_0020_MR3
Processado 350/373: OAS2_0078_MR3
Processado 350/373: OAS2_0078_MR3
Processado 373/373: OAS2_0161_MR2

Processamento concluído. 373 imagens analisadas.

Pré-visualização dos dados de features extraídos:
   Ventricle_Area  Ventricle_Perimeter  Ventricle_Circularity  \
0          3140.0           503.493470               0.155651   
1          3999.0           894.571681               0.062796   
2          3733.0           706.139177               0.094078   
3          4058.0           675.346284               0.

In [5]:
print("Carregando dados demográficos para fusão...")

try:
    # Carregar o CSV
    df_demographic = pd.read_csv(CSV_PATH, sep=';', decimal=',')
    
    if 'MRI ID' not in df_demographic.columns:
         print(f"ERRO: A coluna 'MRI ID' não foi encontrada em {CSV_PATH}")
    elif 'df_features' not in locals() or df_features.empty:
         print("ERRO: O DataFrame 'df_features' está vazio. Execute a Célula 4 primeiro.")
    else:

        # Limpa a coluna 'MRI ID' do CSV, removendo espaços em branco
        df_demographic['MRI ID'] = df_demographic['MRI ID'].str.strip()
        # (A coluna 'MRI ID' do df_features já foi limpa na Célula 4)
        
        # Funde os DataFrames
        df_final = pd.merge(df_demographic, df_features, on='MRI ID', how='left')
        
        print("Fusão realizada com sucesso. Pré-visualização do DataFrame final:")
        print(df_final.head())
        
        # Verificar quantas imagens conseguimos mapear
        mapeados = df_final['Ventricle_Area'].notna().sum()
        total_visitas = len(df_final)
        
        # Esta é a verificação final:
        if mapeados > 0:
            print(f"\nSUCESSO! {mapeados} de {total_visitas} visitas/linhas foram mapeadas com features de imagem.")
        else:
            print(f"\nFALHA NO MERGE. {mapeados} de {total_visitas} visitas/linhas foram mapeadas.")
            print("Verifique se os IDs realmente existem em ambos os datasets.")

except FileNotFoundError:
    print(f"ERRO: Arquivo demográfico não encontrado em {CSV_PATH}")
except Exception as e:
    print(f"ERRO durante a fusão: {e}")

Carregando dados demográficos para fusão...
Fusão realizada com sucesso. Pré-visualização do DataFrame final:
  Subject ID         MRI ID        Group  Visit  MR Delay M/F Hand  Age  EDUC  \
0  OAS2_0001  OAS2_0001_MR1  Nondemented      1         0   M    R   87    14   
1  OAS2_0001  OAS2_0001_MR2  Nondemented      2       457   M    R   88    14   
2  OAS2_0002  OAS2_0002_MR1     Demented      1         0   M    R   75    12   
3  OAS2_0002  OAS2_0002_MR2     Demented      2       560   M    R   76    12   
4  OAS2_0002  OAS2_0002_MR3     Demented      3      1895   M    R   80    12   

   SES  ...  CDR  eTIV   nWBV    ASF  Ventricle_Area  Ventricle_Perimeter  \
0  2.0  ...  0.0  1987  0.696  0.883          4717.0           686.760497   
1  2.0  ...  0.0  2004  0.681  0.876          4943.0           648.997041   
2  NaN  ...  0.5  1678  0.736  1.046          4809.0           870.264069   
3  NaN  ...  0.5  1738  0.713  1.010          4566.0           609.819372   
4  NaN  ...  0.5  

In [6]:
if 'df_final' in locals():
    output_path_full = '../database/features_full.csv'
    output_path_identifiers = '../database/features_identifiers.csv'

    # Salvar o DataFrame completo
    df_final.to_csv(output_path_full, index=False, sep=';', decimal=',')

    # Criar e salvar o DataFrame apenas com identificadores e features
    identifier_columns = ['Subject ID', 'MRI ID', 'Group']
    feature_columns = [col for col in df_final.columns if col not in identifier_columns]
    df_identifiers = df_final[identifier_columns + feature_columns]
    df_identifiers.to_csv(output_path_identifiers, index=False, sep=';', decimal=',')

    print(f"\nPlanilhas finais salvas com sucesso:\n- Completa: {output_path_full}\n- Identificadores + Features: {output_path_identifiers}")
else:
    print("\nERRO: DataFrame 'df_final' não foi criado. As planilhas não foram salvas.")


Planilhas finais salvas com sucesso:
- Completa: ../database/features_full.csv
- Identificadores + Features: ../database/features_identifiers.csv
